In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import warnings
from astropy.wcs import WCS
from astropy.io import fits
from scipy.optimize import curve_fit, root_scalar
from spectral_cube import SpectralCube
from lmfit import Model, Parameters
from scipy.interpolate import interp1d
import aplpy

In [3]:
warnings.filterwarnings('ignore')

# Constants
etamb = 0.89
etaf = 1.0
nu11 = 23.6944955e9
nu22 = 23.7226333e9
h = 6.62606896e-34
kB = 1.3806504e-23
Tbg = 2.73
c = 2.99792458e8
To = 41.5
epsilon = 8.854187817e-12
dipole = 298117.06e6
C_moment = 186726.36e6
mu = 1.476 * 3.336e-30
mu11 = mu ** 2 * (1. / 2.)
Einstein_A = (16. * np.pi ** 3 / (3. * epsilon * h * c ** 3)) * nu11 ** 3 * abs(mu11)
J_Tbg = h * nu11 / kB * (1. / (np.exp(h * nu11 / (kB * Tbg)) - 1))
etac = etamb * etaf
mNH3=17.03      #amu
mH=1.00794      #amu
amu=1.66053886e-27  #kg

In [4]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x - cen)**2 / (2 * wid**2))

def quad_func(x, amp0, vel0, sigma0, tau0): # (1,1) Single-gaussian profile
    amp = [0.226, 0.273, 1.0, 0.277, 0.219] # Relative expected amplitudes of five-finger structure
    vel = [-19.503, -7.594, 0.0, 7.599, 19.493] # Relative expected velocities of five-finger structure
    emission_profile = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0)) # Amplitude of gaussian is ratio of main/sattelite line amplitudes
        emission_profile += gaussian(x, Tq, vel0 + veli, sigma0)
    return amp0*emission_profile

def quad_func2c(x, amp0, vel0, sigma0, tau0, amp1, vel1, sigma1, tau1): # (1,1) Double-gaussian
    amp = [0.226, 0.273, 1.0, 0.277, 0.219]
    vel = [-19.503, -7.594, 0.0, 7.599, 19.493]

    emission_profile0 = np.zeros_like(x)
    emission_profile1 = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq0=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        Tq1=(1-np.exp(-ampi*tau1))/(1-np.exp(-tau1))
        emission_profile0 += gaussian(x, Tq0, vel0 + veli, sigma0)
        emission_profile1 += gaussian(x, Tq1, vel1 + veli, sigma1)
    return amp0*emission_profile0+amp1*emission_profile1

def quad_func22(x, amp0, vel0, sigma0, tau0): # (2,2) Single-gaussian profile
    amp = [0.063, 0.039, 1.0, 0.092, 0.063]
    vel = [-26.023, -16.368, 0.0, 16.368, 26.023]

    emission_profile = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        emission_profile += gaussian(x, Tq, vel0 + veli, sigma0)
    return amp0*emission_profile

def quad_func222c(x, amp0, vel0, sigma0, tau0, amp1, vel1, sigma1, tau1): # (2,2) Double-gaussian profile
    amp = [0.063, 0.039, 1.0, 0.092, 0.063]
    vel = [-26.023, -16.368, 0.0, 16.368, 26.023]

    emission_profile0 = np.zeros_like(x)
    emission_profile1 = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq0=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        Tq1=(1-np.exp(-ampi*tau1))/(1-np.exp(-tau1))
        emission_profile0 += gaussian(x, Tq0, vel0 + veli, sigma0)
        emission_profile1 += gaussian(x, Tq1, vel1 + veli, sigma1)
    return amp0*emission_profile0+amp1*emission_profile1

def solve_Tk(Tr): # Solve for kinetic temperature Tk given rotational temperature Tr
    if np.isnan(Tr):
        return np.nan

    def KineticTemp(Tk):
        if Tk <= 0:
            return 1e6  # prevent log or exp issues
        return Tk / (1. + ((Tk / To) * np.log(1. + (0.6 * np.exp(-15.7 / Tk))))) - Tr

    try:
        sol = root_scalar(KineticTemp, bracket=[1, 500], method='brentq')
        return sol.root if sol.converged else np.nan
    except Exception:
        return np.nan

In [ ]:
# File paths
path = '/users/hfwest/GBO-REU/'
datadir = path + 'RAMPS-Results/'
suffixout='_test'
Cube11 = SpectralCube.read('/home/scratch/hfwest/RAMPS-Data/L13_NH3_1-1_cube.fits')

In [12]:
# Grab the middle channel as a 2D slice
slice2d11 = Cube11[Cube11.spectral_axis.size // 2]

# Get the WCS from the 2D slice
wcs_3d11 = Cube11.wcs
wcs_2d11 = slice2d11.wcs

# Convert to header
header_3d11 = wcs_3d11.to_header()
header_2d11 = wcs_2d11.to_header()

vmargin = 50.
threshold = 5   # N sigma level to fit above
thresholdlower = 3   # N sigma level to fit secondary/tertiary components above
vl = -2000 * u.km / u.s    # Within bounds of observed emission TODO: adjust if adapting to RAMPS data
vh = 5000 * u.km / u.s
rmsvl = 6000 * u.km / u.s    # Outside bounds of observed emission TODO: adjust if adapting to RAMPS data
rmsvh = 10000 * u.km / u.s
# Reference pixel (adjusted for Python's 0-indexing)
xpeak = 83      # v.strong
ypeak = 79
xpeak = 105     # clear two-component area
ypeak = 62
xpeak = 98      # hopefully simple pixel
ypeak = 84
xpeak = 52     # probable non-detection
ypeak = 115
xpeak = 91     # potential three-component area
ypeak = 74
ref_pixx = xpeak - 1
ref_pixy = ypeak - 1

In [13]:
Cube11 = Cube11.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=nu11*u.Hz)

Cube11_slab = Cube11.spectral_slab(vl, vh)
RMS11_slab = Cube11.spectral_slab(rmsvl, rmsvh)
vel_axis11 = Cube11_slab.spectral_axis # Range of velocities considered
cube11_data = Cube11_slab.unmasked_data[:].value
rms11_data = RMS11_slab.unmasked_data[:].value

ny, nx = Cube11.shape[1], Cube11.shape[2]
nz = vel_axis11.shape[0]

In [15]:
TMaxMap11 = np.zeros((ny, nx))
TauMap11 = np.zeros((ny, nx))
SigmaMap11 = np.zeros((ny, nx))
VelMap11 = np.zeros((ny, nx))
TempRotMap = np.zeros((ny, nx))
TempExMap = np.zeros((ny, nx))
TempKinMap = np.zeros((ny, nx))
FFMap = np.zeros((ny, nx))
NColMap = np.zeros((ny, nx))
Sigma_Therm_Map = np.zeros((ny, nx))
Sigma_Turb_Map11 = np.zeros((ny, nx))
TMaxMap11Comp1 = np.zeros((ny, nx))
SigmaMap11Comp1 = np.zeros((ny, nx))
VelMap11Comp1 = np.zeros((ny, nx))
ResidMap11 = np.zeros((ny, nx))
ResidCube11 = np.zeros((nz, ny, nx))
RedChiMap11 = np.zeros((ny, nx))

modelq = Model(quad_func)
modelq2c = Model(quad_func2c)
modelq22 = Model(quad_func22)
modelq222c = Model(quad_func222c)

In [ ]:
for j in range(ny):
    for i in range(nx):
        spec11 = cube11_data[:, j, i] # Full spectrum at pixel (i,j)
        rms11 = np.std(rms11_data[:, j, i])  # emission-free region of spectrum gives baseline value
        testmaxindex11 = np.argmax(spec11)          # Index of velocity corresponding to largest peak in (1,1) spectrum
        testvel_peak11 = vel_axis11[testmaxindex11] # Velocity corresponding to largest peak in (1,1) spectrum
        amp011 = spec11[testmaxindex11]             # Height of largest peak in (1,1) spectrum

        if amp011 > threshold * rms11: # If peak of (1,1) spectrum is greater than 5 sigma above the baseline

            #RMS is fairly uniform at around 0.011 across most of the map
            #The overlapping region has an RMS of ~0.013, maxing out at ~0.02
            
            mask11 = (vel_axis11.value > testvel_peak11.value - vmargin) & (vel_axis11.value < testvel_peak11.value + vmargin)
            # Value mask: velocities within |vmargin| distance of the peak velocity. In this case, vmargin = 50 km/s
            vel_axis11_clip = vel_axis11[mask11]
            spec11_clip = spec11[mask11]
            # velocities and spectra for masked range (+- vmargin from peak) at this pixel


            # Single gaussian solution parameters:
            paramsq11 = Parameters()
            paramsq11.add('amp0', value=amp011, min=threshold * rms11)
            paramsq11.add('vel0', value=testvel_peak11.value)
            paramsq11.add('sigma0', value=1.0, min=0.01)
            paramsq11.add('tau0', value=1.5, max=20, min=0.1)

            # Double gaussian solution parameters:
            paramsq112c = Parameters()
            paramsq112c.add('amp0', value=amp011, min=threshold * rms11)
            paramsq112c.add('vel0', value=testvel_peak11.value)
            paramsq112c.add('sigma0', value=1.0, min=0.01)
            paramsq112c.add('tau0', value=1.5, max=20, min=0.01)
            paramsq112c.add('amp1', value=amp011*0.8, min=thresholdlower * rms11)
            paramsq112c.add('vel1', value=testvel_peak11.value+5)
            paramsq112c.add('sigma1', value=1.0, min=0.01)
            paramsq112c.add('tau1', value=1.0, max=20, min=0.1)

            resultq112c = modelq2c.fit(spec11_clip, paramsq112c, x=vel_axis11_clip.value)
            if resultq112c.params['amp0'].value < resultq112c.params['amp1'].value:
                SigmaMap11[j,i] = resultq112c.params['sigma1'].value
                VelMap11[j,i] = resultq112c.params['vel1'].value
                TMaxMap11Comp1[j,i] = resultq112c.params['amp1'].value
                SigmaMap11Comp1[j,i] = resultq112c.params['sigma1'].value
                VelMap11Comp1[j,i] = resultq112c.params['vel1'].value
            else:
                SigmaMap11[j,i] = resultq112c.params['sigma0'].value
                VelMap11[j,i] = resultq112c.params['vel0'].value
                TMaxMap11Comp1[j,i] = resultq112c.params['amp0'].value
                SigmaMap11Comp1[j,i] = resultq112c.params['sigma0'].value
                VelMap11Comp1[j,i] = resultq112c.params['vel0'].value
            InterpResid11 = interp1d(vel_axis11_clip, resultq112c.residual, bounds_error=False, fill_value=0.0)
            ResidCube11[:,j,i] = InterpResid11(vel_axis11)
            ResidMap11[j,i] = np.sqrt(np.mean(resultq112c.residual**2))
            RedChiMap11[j,i] = resultq112c.redchi

            Tau11_main = resultq112c.params['tau0'].value+resultq112c.params['tau1'].value
            TMax11 = np.nanmax([resultq112c.params['amp0'].value,resultq112c.params['amp1'].value])
            TauMap11[j,i] = Tau11_main * 1.995 # (1,1 quadrupoles are 0.226, 0.273, 0.277, 0.219, sum = 0.995 of the main height, so the total is 1.995 and the ratio of the main line to the total is 1/1.995)
            TMaxMap11[j,i] = TMax11

        B=1.-np.exp(-1.*TauMap11[j,i]/1.995)
        TempExMap[j,i] = h*nu11/(kB*np.log((h*nu11/kB)*(1./(TMaxMap11[j,i]/(etac*B)+J_Tbg)+1)))
        Sigma_Turb_Map11[j,i]=((SigmaMap11[j,i]**2.)-(Sigma_Therm_Map[j,i]**2.))**0.5

In [ ]:
TMaxMap11 = fits.PrimaryHDU(TMaxMap11, header=header_2d11)
TMaxMap11.writeto(datadir + 'TMaxMap11' + suffixout + '.fits', overwrite=True)
TauMap11 = fits.ImageHDU(TauMap11, header=header_2d11)
TauMap11.writeto(datadir + 'TauMap11' + suffixout + '.fits', overwrite=True)
SigmaMap11 = fits.ImageHDU(SigmaMap11, header=header_2d11)
SigmaMap11.writeto(datadir + 'SigmaMap11' + suffixout + '.fits', overwrite=True)
VelMap11 = fits.ImageHDU(VelMap11, header=header_2d11)
VelMap11.writeto(datadir + 'VelMap11' + suffixout + '.fits', overwrite=True)
TempExMap = fits.ImageHDU(TempExMap, header=header_2d11)
TempExMap.writeto(datadir + 'TempExMap' + suffixout + '.fits', overwrite=True)
TMaxMap11Comp1 = fits.ImageHDU(TMaxMap11Comp1, header=header_2d11)
TMaxMap11Comp1.writeto(datadir + 'TMaxMap11Comp1' + suffixout + '.fits', overwrite=True)
SigmaMap11Comp1 = fits.ImageHDU(SigmaMap11Comp1, header=header_2d11)
SigmaMap11Comp1.writeto(datadir + 'SigmaMap11Comp1' + suffixout + '.fits', overwrite=True)
VelMap11Comp1 = fits.ImageHDU(VelMap11Comp1, header=header_2d11)
VelMap11Comp1.writeto(datadir + 'VelMap11Comp1' + suffixout + '.fits', overwrite=True)
ResidMap11 = fits.ImageHDU(ResidMap11, header=header_2d11)
ResidMap11.writeto(datadir + 'ResidMap11' + suffixout + '.fits', overwrite=True)
ResidCube11 = fits.PrimaryHDU(ResidCube11, header=header_3d11)
ResidCube11.writeto(datadir + 'ResidCube11' + suffixout + '.fits', overwrite=True)
RedChiMap11 = fits.ImageHDU(RedChiMap11, header=header_2d11)
RedChiMap11.writeto(datadir + 'RedChiMap11' + suffixout + '.fits', overwrite=True)